# Using GPR to predict Renewable Energy Investments

In [52]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, widgets
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt

### Graph helper functions

In [2]:
def update_layout_of_graph(fig: go.Figure,title: str = 'Plot')->go.Figure:
    fig.update_layout(
        width=800,
        height=600,
        autosize=False,
        plot_bgcolor='rgba(0,0,0,0)',
        title=title,
        
    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                      xaxis_title = 'input values',
                      yaxis_title = 'output values',
                      legend=dict(yanchor="top",
                                  y=0.9,
                                  xanchor="right",
                                  x=0.95),
                      title={
                          'x': 0.5,
                          'xanchor': 'center'
                      })
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
    return fig

In [3]:
 def line_scatter(
    visible: bool = True,
    x_lines: np.array = np.array([]),
    y_lines: np.array = np.array([]),
    name_line: str = 'Predicted function',
    showlegend: bool = True,
) -> go.Scatter:
    # Adding the lines
    return go.Scatter(
        visible=visible,
        line=dict(color="blue", width=2),
        x=x_lines,
        y=y_lines,
        name=name_line,
        showlegend= showlegend
    )

In [4]:
def dot_scatter(
    visible: bool = True,
    x_dots: np.array = np.array([]),
    y_dots: np.array = np.array([]),
    name_dots: str = 'Observed points',
    showlegend: bool = True
) -> go.Scatter:
    # Adding the dots
    return go.Scatter(
        x=x_dots,
        visible=visible,
        y=y_dots,
        mode="markers",
        name=name_dots,
        marker=dict(color='red', size=8),
        showlegend=showlegend
    )

In [5]:
def uncertainty_area_scatter(
        visible: bool = True,
        x_lines: np.array = np.array([]),
        y_upper: np.array = np.array([]),
        y_lower: np.array = np.array([]),
        name: str = "mean plus/minus standard deviation",
) -> go.Scatter:

    return go.Scatter(
        visible=visible,
        x=np.concatenate((x_lines, x_lines[::-1])),  # x, then x reversed
        # upper, then lower reversed
        y=np.concatenate((y_upper, y_lower[::-1])),
        fill='toself',
        fillcolor='rgba(189,195,199,0.5)',
        line=dict(color='rgba(200,200,200,0)'),
        hoverinfo="skip",
        showlegend=True,
        name= name,
    )

In [6]:
def add_slider_GPR(figure: go.Figure, parameters):
    figure.data[0].visible = True
    figure.data[1].visible = True

    # Create and add slider
    steps = []
    for i in range(int((len(figure.data) - 1) / 2)):
        step = dict(
            method="update",
            label=f'{parameters[i]: .2f}',
            args=[{
                "visible": [False] * (len(figure.data) - 1) + [True]
            }],
        )
        step["args"][0]["visible"][2 *
                                   i] = True  # Toggle i'th trace to "visible"
        step["args"][0]["visible"][2 * i + 1] = True
        steps.append(step)

    sliders = [dict(
        active=0,
        pad={"t": 50},
        steps=steps,
    )]
    figure.update_layout(sliders=sliders, )
    return figure

In [7]:
def add_slider_to_function(figure:go.Figure, parameters):
    figure.data[0].visible = True

    # Create and add slider
    steps = []
    for i in range(len(figure.data)):
        step = dict(
            method="update",
            label=f'{parameters[i]: .2f}',
            args=[{
                "visible": [False] *len(figure.data) 
            }],
        )
        step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active=0,
        pad={"t": 50},
        steps=steps,
    )]
    figure.update_layout(sliders=sliders, )
    return figure

In [8]:
class SquaredExponentialKernel:
    def __init__(self, sigma_f: float = 1, length: float = 1):
        self.sigma_f = sigma_f
        self.length = length

    def __call__(self, argument_1: np.array, argument_2: np.array) -> float:
        return float(self.sigma_f *
                     np.exp(-(np.linalg.norm(argument_1 - argument_2)**2) /
                            (2 * self.length**2)))



In [9]:
x_lines = np.arange(-10, 10, 0.1)
kernel = SquaredExponentialKernel(length=1)

fig0 = go.FigureWidget(data=[
    line_scatter(
        x_lines=x_lines,
        y_lines=np.array([kernel(x, 0) for x in x_lines]),
    )
])

fig0 = update_layout_of_graph(fig0, title='Squared exponential kernel')


@interact(length=(0.1, 3, 0.1), argument_2=(-10, 10, 0.1))
def update(length=1, argument_2=0):
    with fig0.batch_update():
        kernel = SquaredExponentialKernel(length=length)
        fig0.data[0].y = np.array([kernel(x, argument_2) for x in x_lines])


fig0

interactive(children=(FloatSlider(value=1.0, description='length', max=3.0, min=0.1), FloatSlider(value=0.0, d…

FigureWidget({
    'data': [{'line': {'color': 'blue', 'width': 2},
              'name': 'Predicted function',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'a7617d5e-d3cf-4609-a9af-d247d1ba8e4e',
              'visible': True,
              'x': array([-1.00000000e+01, -9.90000000e+00, -9.80000000e+00, -9.70000000e+00,
                          -9.60000000e+00, -9.50000000e+00, -9.40000000e+00, -9.30000000e+00,
                          -9.20000000e+00, -9.10000000e+00, -9.00000000e+00, -8.90000000e+00,
                          -8.80000000e+00, -8.70000000e+00, -8.60000000e+00, -8.50000000e+00,
                          -8.40000000e+00, -8.30000000e+00, -8.20000000e+00, -8.10000000e+00,
                          -8.00000000e+00, -7.90000000e+00, -7.80000000e+00, -7.70000000e+00,
                          -7.60000000e+00, -7.50000000e+00, -7.40000000e+00, -7.30000000e+00,
                          -7.20000000e+00, -7.10000000e+00, -7.00000000

In [53]:
 # Helper function to calculate the respective covariance matrices
def cov_matrix(x1, x2, cov_function) -> np.array:
    return np.array([[cov_function(a, b) for a in x1] for b in x2])

### Define the main GPR class

In [11]:
class GPR:
    def __init__(self,
                 data_x: np.array,
                 data_y: np.array,
                 covariance_function=SquaredExponentialKernel(),
                 white_noise_sigma: float = 0):
        self.noise = white_noise_sigma
        self.data_x = data_x
        self.data_y = data_y
        self.covariance_function = covariance_function

        # Store the inverse of covariance matrix of input (+ machine epsilon on diagonal) since it is needed for every prediction
        self._inverse_of_covariance_matrix_of_input = np.linalg.inv(
            cov_matrix(data_x, data_x, covariance_function) +
            (3e-7 + self.noise) * np.identity(len(self.data_x)))

        self._memory = None

    # function to predict output at new input values. Store the mean and covariance matrix in memory.

    def predict(self, at_values: np.array) -> np.array:
        k_lower_left = cov_matrix(self.data_x, at_values,
                                  self.covariance_function)
        k_lower_right = cov_matrix(at_values, at_values,
                                   self.covariance_function)

        # Mean.
        mean_at_values = np.dot(
            k_lower_left,
            np.dot(self.data_y,
                   self._inverse_of_covariance_matrix_of_input.T).T).flatten()

        # Covariance.
        cov_at_values = k_lower_right - \
            np.dot(k_lower_left, np.dot(
                self._inverse_of_covariance_matrix_of_input, k_lower_left.T))

        # Adding value larger than machine epsilon to ensure positive semi definite
        cov_at_values = cov_at_values + 3e-7 * np.ones(
            np.shape(cov_at_values)[0])

        var_at_values = np.diag(cov_at_values)

        self._memory = {
            'mean': mean_at_values,
            'covariance_matrix': cov_at_values,
            'variance': var_at_values
        }
        return mean_at_values

In [12]:
np.random.seed(1)

#Importing the CSV File
df = pd.read_csv("Investment_Data_Train.csv")

#Removing Rows with Null Values
df['TotalNumberofInvestments'].replace('', np.nan, inplace=True)
df.dropna(subset=['TotalNumberofInvestments'], inplace=True)

#Renaming Columns for Convinience
df.rename(columns = {'CO2 Emissions (Mmt)':'MMT'}, inplace = True)

#Removing Unnecessary Columns
df = df[['MSN', 'StateCode', 'Year', 'Amount', 'MMT', 'TotalNumberofInvestments', 'TotalAmountofAssistance']]

df

y = np.asarray(df['TotalAmountofAssistance'])
res = []
[res.append(x) for x in y if x not in res]

df


,MSN,StateCode,Year,Amount,MMT,TotalNumberofInvestments,TotalAmountofAssistance
0,BDFDB,AK,2015,21.0,35.027804,16.0,3345612.0
1,BDPRP,AK,2015,4.0,35.027804,16.0,3345612.0
2,BFFDB,AK,2015,21.0,35.027804,16.0,3345612.0
3,BFPRP,AK,2015,4.0,35.027804,16.0,3345612.0
4,CLPRB,AK,2015,17747.0,35.027804,16.0,3345612.0
...,...,...,...,...,...,...,...
7579,WDPRB,WY,2019,4969.0,59.096649,32.0,570570.0
7580,WDTCB,WY,2019,4969.0,59.096649,32.0,570570.0
7581,WSTCB,WY,2019,0.0,59.096649,32.0,570570.0
7582,WWPRB,WY,2019,4969.0,59.096649,32.0,570570.0


In [13]:
# Organizing the Table into Desired Format using Pivot Tables
multi_variables = df
multi_variables['TotalNumberofInvestments'].replace('', np.nan, inplace=True)
multi_variables.dropna(subset=['TotalNumberofInvestments'], inplace=True)

multi_variables = df[['StateCode', 'Year', 'MSN', 'Amount', 'MMT']].drop_duplicates()

multi_variables = multi_variables.pivot(index=['StateCode', 'Year', 'MMT'], columns='MSN', values='Amount')

multi_variables.reset_index()

MSN,StateCode,Year,MMT,BDFDB,BDPRP,BFFDB,BFPRP,CLPRB,CLPRK,CLPRP,...,REPRB,SOTCB,TEPRB,TETCB,WDEXB,WDPRB,WDTCB,WSTCB,WWPRB,WYTCB
0,AK,2015,35.027804,21.0,4.0,21.0,4.0,17747.0,15.073,1177.0,...,23807.0,3.0,1429942.0,614242.0,0.0,6865.0,6865.0,622.0,7488.0,1488.0
1,AK,2016,33.405611,27.0,5.0,27.0,5.0,13942.0,14.957,932.0,...,25155.0,14.0,1435363.0,593662.0,0.0,7636.0,7636.0,412.0,8048.0,1563.0
2,AK,2017,33.729130,29.0,5.0,29.0,5.0,14365.0,14.978,959.0,...,23620.0,20.0,1454196.0,601492.0,0.0,6501.0,6501.0,436.0,6937.0,1305.0
3,AK,2018,34.514589,15.0,3.0,15.0,3.0,13752.0,15.253,902.0,...,24158.0,28.0,1410604.0,604644.0,0.0,6910.0,6910.0,456.0,7366.0,1411.0
4,AK,2019,34.259440,0.0,0.0,0.0,0.0,14867.0,15.252,975.0,...,22754.0,40.0,1370342.0,605689.0,0.0,6418.0,6418.0,386.0,6803.0,1270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,WY,2015,64.092863,0.0,0.0,1192.0,209.0,6538240.0,17.399,375773.0,...,49887.0,27.0,9080777.0,507121.0,0.0,4904.0,4904.0,2.0,4905.0,35009.0
246,WY,2016,61.259993,0.0,0.0,0.0,0.0,5169939.0,17.394,297218.0,...,54565.0,33.0,7518440.0,504984.0,0.0,4357.0,4357.0,2.0,4358.0,40522.0
247,WY,2017,62.796487,0.0,0.0,0.0,0.0,5516757.0,17.433,316454.0,...,55916.0,45.0,7794586.0,534840.0,0.0,5049.0,5049.0,0.0,5049.0,39806.0
248,WY,2018,63.846081,0.0,0.0,0.0,0.0,5315953.0,17.476,304188.0,...,51483.0,67.0,7711992.0,557604.0,0.0,4934.0,4934.0,0.0,4934.0,36936.0


In [45]:
msn1 = multi_variables.reset_index()['BDFDB'].values
msn2 = multi_variables.reset_index()['BDPRP'].values
msn3 = multi_variables.reset_index()['BFFDB'].values
msn4 = multi_variables.reset_index()['BFPRP'].values
msn5 = multi_variables.reset_index()['CLPRB'].values
msn6 = multi_variables.reset_index()['CLPRK'].values
msn7 = multi_variables.reset_index()['CLPRP'].values
msn8 = multi_variables.reset_index()['COPRK'].values
msn9 = multi_variables.reset_index()['EMFDB'].values
msn10 = multi_variables.reset_index()['ENPRP'].values
msn11 = multi_variables.reset_index()['GETCB'].values
msn12 = multi_variables.reset_index()['HYTCB'].values
msn13 = multi_variables.reset_index()['NCPRB'].values
msn14 = multi_variables.reset_index()['NGMPB'].values
msn15 = multi_variables.reset_index()['NGMPK'].values
msn16 = multi_variables.reset_index()['NGMPP'].values
msn17 = multi_variables.reset_index()['NUETB'].values
msn18 = multi_variables.reset_index()['PAPRB'].values
msn19 = multi_variables.reset_index()['PAPRP'].values
msn20 = multi_variables.reset_index()['REPRB'].values
msn21 = multi_variables.reset_index()['SOTCB'].values
msn22 = multi_variables.reset_index()['TEPRB'].values
msn23 = multi_variables.reset_index()['TETCB'].values
msn24 = multi_variables.reset_index()['WDEXB'].values
msn25 = multi_variables.reset_index()['WDPRB'].values
msn26 = multi_variables.reset_index()['WDTCB'].values
msn27 = multi_variables.reset_index()['WSTCB'].values
msn28 = multi_variables.reset_index()['WWPRB'].values
msn29 = multi_variables.reset_index()['WYTCB'].values

# x = np.asarray([msn1, msn2, msn3, msn4, msn5, msn6, msn7, msn8, msn9, msn10, msn11, msn12, msn13, msn14, msn15, msn16, msn17, msn18, msn19, msn20, msn21, msn22, msn23, msn24, msn25, msn26, msn27, msn28, msn29]).T
x = msn3
#print(x)
multi_variables


MSN                       BDFDB  BDPRP   BFFDB  BFPRP      CLPRB   CLPRK  \
StateCode Year MMT                                                         
AK        2015 35.027804   21.0    4.0    21.0    4.0    17747.0  15.073   
          2016 33.405611   27.0    5.0    27.0    5.0    13942.0  14.957   
          2017 33.729130   29.0    5.0    29.0    5.0    14365.0  14.978   
          2018 34.514589   15.0    3.0    15.0    3.0    13752.0  15.253   
          2019 34.259440    0.0    0.0     0.0    0.0    14867.0  15.252   
...                         ...    ...     ...    ...        ...     ...   
WY        2015 64.092863    0.0    0.0  1192.0  209.0  6538240.0  17.399   
          2016 61.259993    0.0    0.0     0.0    0.0  5169939.0  17.394   
          2017 62.796487    0.0    0.0     0.0    0.0  5516757.0  17.433   
          2018 63.846081    0.0    0.0     0.0    0.0  5315953.0  17.476   
          2019 59.096649    0.0    0.0     0.0    0.0  4828513.0  17.437   

MSN                          CLPRP  COPRK   EMFDB  ENPRP  ...    REPRB  \
StateCode Year MMT                                        ...            
AK        2015 35.027804    1177.0  5.717     0.0    0.0  ...  23807.0   
          2016 33.405611     932.0  5.722     0.0    0.0  ...  25155.0   
          2017 33.729130     959.0  5.723     0.0    0.0  ...  23620.0   
          2018 34.514589     902.0  5.706     0.0    0.0  ...  24158.0   
          2019 34.259440     975.0  5.698     0.0    0.0  ...  22754.0   
...                            ...    ...     ...    ...  ...      ...   
WY        2015 64.092863  375773.0  5.717  1192.0  209.0  ...  49887.0   
          2016 61.259993  297218.0  5.722     0.0    0.0  ...  54565.0   
          2017 62.796487  316454.0  5.723     0.0    0.0  ...  55916.0   
          2018 63.846081  304188.0  5.706     0.0    0.0  ...  51483.0   
          2019 59.096649  276912.0  5.698     0.0    0.0  ...  53228.0   

MSN                        SOTCB      TEPRB     TETCB  WDEXB   WDPRB   WDTCB  \
StateCode Year MMT                                                             
AK        2015 35.027804     3.0  1429942.0  614242.0    0.0  6865.0  6865.0   
          2016 33.405611    14.0  1435363.0  593662.0    0.0  7636.0  7636.0   
          2017 33.729130    20.0  1454196.0  601492.0    0.0  6501.0  6501.0   
          2018 34.514589    28.0  1410604.0  604644.0    0.0  6910.0  6910.0   
          2019 34.259440    40.0  1370342.0  605689.0    0.0  6418.0  6418.0   
...                          ...        ...       ...    ...     ...     ...   
WY        2015 64.092863    27.0  9080777.0  507121.0    0.0  4904.0  4904.0   
          2016 61.259993    33.0  7518440.0  504984.0    0.0  4357.0  4357.0   
          2017 62.796487    45.0  7794586.0  534840.0    0.0  5049.0  5049.0   
          2018 63.846081    67.0  7711992.0  557604.0    0.0  4934.0  4934.0   
          2019 59.096649  1688.0  7148285.0  546671.0    0.0  4969.0  4969.0   

MSN                       WSTCB   WWPRB    WYTCB  
StateCode Year MMT                                
AK        2015 35.027804  622.0  7488.0   1488.0  
          2016 33.405611  412.0  8048.0   1563.0  
          2017 33.729130  436.0  6937.0   1305.0  
          2018 34.514589  456.0  7366.0   1411.0  
          2019 34.259440  386.0  6803.0   1270.0  
...                         ...     ...      ...  
WY        2015 64.092863    2.0  4905.0  35009.0  
          2016 61.259993    2.0  4358.0  40522.0  
          2017 62.796487    0.0  5049.0  39806.0  
          2018 63.846081    0.0  4934.0  36936.0  
          2019 59.096649    0.0  4969.0  37070.0  

[250 rows x 29 columns]

In [46]:
x_values = x[6:10]
print(x_values)
y_values = res[6:10]
print(y_values)

[1906. 1585. 1652. 1494.]
[1340702.0, 1452666.0, 3527420.0, 6155856.0]


In [34]:
x = np.arange(-1,7,0.1) 

In [55]:
 def plot_GPR(data_x, data_y, model, x, visible=True) -> list:
    mean = model.predict(x)

    std = np.sqrt(model._memory['variance'])
    data = []

    for i in range(1, 4):
        data.append(
            uncertainty_area_scatter(
                x_lines=x,
                y_lower=mean - i * std,
                y_upper=mean + i * std,
                name=f"mean plus/minus {i}*standard deviation",
                visible=visible))

    data.append(line_scatter(x_lines=x, y_lines=mean, visible=visible))
    data.append(dot_scatter(x_dots=data_x, y_dots=data_y, visible=visible))
    return data

### Initiate Training

In [54]:
model = GPR(x_values, y_values)
data = plot_GPR(data_x=x_values, data_y=y_values, x=x, model=model)
fig4 = go.Figure(data=data)
fig4 = update_layout_of_graph(fig=fig4,
title='GPR with length 1, sigma 0 and noise 0')

fig4.show()

In [37]:
# model = GPR(x_values, y_values)

# mean = model.predict(x)
# covariance_matrix = model._memory['covariance_matrix']

# fig1 = go.FigureWidget(data=[dot_scatter(x_dots=x_values, y_dots=y_values)])
# fig1 = update_layout_of_graph(
#     fig1,
#     title='Random drawings (i.e. random functions) of the Gaussian process')

# button = widgets.Button(description='Add random drawing')


# def update(_):
#     with fig1.batch_update():
#         fig1.add_trace(
#             line_scatter(x_lines=x,
#                          y_lines=np.random.multivariate_normal(
#                              mean, covariance_matrix),
#                          name_line='random function',
#                          showlegend=False))
#         fig1.add_trace(
#             dot_scatter(x_dots=x_values, y_dots=y_values, showlegend=False))


# button.on_click(update)
# widgets.VBox([fig1, button])

In [38]:
# model = GPR(x_values[:1], y_values[:1])

# data = plot_GPR(data_x=x_values[:1], data_y=y_values[:1], x=x, model=model)

# fig2 = go.FigureWidget(data=data)
# fig2 = update_layout_of_graph(fig2, title='Prediction (i.e. mean) of GPR')

# button = widgets.Button(description='Add data point')

# number_of_points = 1

# # we update the y values of our 4 scatter plots 
# def update(_):
#     with fig2.batch_update():
#         global number_of_points
#         if number_of_points < len(x_values):
#             number_of_points += 1
#             model = GPR(x_values[:number_of_points],
#                         y_values[:number_of_points])
#             mean = model.predict(x)
#             fig2.data[3].y = mean
#             for i in range(1, 4):
#                 y_upper = mean + i * np.sqrt(model._memory['variance'])
#                 y_lower = mean - i * np.sqrt(model._memory['variance'])
#                 fig2.data[i - 1].y = np.concatenate((y_upper, y_lower[::-1]))

#             fig2.data[4].x = x_values[:number_of_points]
#             fig2.data[4].y = y_values[:number_of_points]


# button.on_click(update)
# widgets.VBox([fig2, button])

In [39]:
# model = GPR(x_values,
#             y_values,
#             covariance_function=SquaredExponentialKernel(length=0.5),
#             white_noise_sigma=0.1)

# data = plot_GPR(x_values, y_values, model=model, x=x)

# fig3 = go.FigureWidget(data=data)


# # we update the y values of our 4 scatter plots 
# @interact(sigma=(0.01, 3, 0.01), length=(0.01, 3, 0.01), noise=(0, 3, 0.1))
# def update(sigma=1, length=0.5, noise=0.1):
#     with fig3.batch_update():
#         model = GPR(x_values,
#                     y_values,
#                     covariance_function=SquaredExponentialKernel(
#                         sigma_f=sigma, length=length),
#                     white_noise_sigma=noise)

#         mean = model.predict(x)
#         y_upper = mean + np.sqrt(model._memory['variance'])
#         y_lower = mean - np.sqrt(model._memory['variance'])

        
#         for i in range(1, 4):
#             y_upper = mean + i * np.sqrt(model._memory['variance'])
#             y_lower = mean - i * np.sqrt(model._memory['variance'])
#             fig3.data[i-1].y = np.concatenate((y_upper, y_lower[::-1]))
        
#         fig3.data[3].y = mean


# fig3 = update_layout_of_graph(
#     fig3, title="GPR with varying length, sigma and white noise")
# fig3 

### Fit and analyze

In [48]:
mean = model.predict(x)
y_upper = mean + np.sqrt(model._memory['variance'])
y_lower = mean - np.sqrt(model._memory['variance'])

print(len(y_upper))
print(len(y_lower))



X_variables = x
y_variables = y_upper

together = zip(X_variables, y_variables)
sum = 0

for i in together:
    sum += ((i[0] - i[1])**2)**0.5

print(sum / (80)**0.5)

sqrt(mean_squared_error(X_variables, y_variables))

250
250
2786693.513690418


481842.709466912

In [51]:
x

array([2.10000e+01, 2.70000e+01, 2.90000e+01, 1.50000e+01, 0.00000e+00,
       1.93300e+03, 1.90600e+03, 1.58500e+03, 1.65200e+03, 1.49400e+03,
       4.42900e+03, 5.37700e+03, 6.76000e+03, 9.49900e+03, 8.58900e+03,
       6.60200e+03, 6.20400e+03, 6.58400e+03, 6.75800e+03, 3.02600e+03,
       3.06010e+04, 3.27650e+04, 3.42400e+04, 3.54650e+04, 3.18770e+04,
       1.98610e+04, 1.94610e+04, 1.98280e+04, 1.97350e+04, 1.87250e+04,
       7.15000e+02, 1.49600e+03, 1.37900e+03, 2.29600e+03, 2.35900e+03,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       6.45000e+02, 5.58000e+02, 8.55000e+02, 1.81800e+03, 9.20000e+02,
       1.45630e+04, 1.71660e+04, 1.61680e+04, 1.51540e+04, 1.69870e+04,
       3.83000e+02, 4.51000e+02, 4.79000e+02, 5.09000e+02, 4.84000e+02,
       5.68234e+05, 5.85659e+05, 6.10395e+05, 6.41146e+05, 6.37409e+05,
       8.87100e+03, 9.20600e+03, 9.36700e+03, 9.32400e+03, 8.94600e+03,
       2.30655e+05, 2.52801e+05, 2.43223e+05, 2.26419e+05, 2.182

In [50]:
y_variables

array([1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.34070160e+06, 1.45266556e+06,
       3.52741894e+06, 6.15585415e+06, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 8.33105256e+05,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
       1.00000015e+00, 1.00000015e+00, 1.00000015e+00, 1.00000015e+00,
      